# Install Environment

In [2]:
import os
import pandas as pd
import torch
import transformers as tr
from datasets import Dataset, load_metric
import numpy as np
from transformers import Trainer
from sklearn.metrics import classification_report, accuracy_score

C:\Users\Shindy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data Importing

## Clean Data

In [ ]:
# GonzaloA data
gonzaloa_train = pd.read_csv(
    "https://github.com/AnasElbattra/Hugging-face/raw/main/FAKE-REAL/DATA/GonzaloA_fake_news/train%20Data.csv"
)  # 24352 row
gonzaloa_train = gonzaloa_train[["text", "label"]]
# gonzaloa_train['label'].replace({0: 'fake', 1: 'real'}, inplace=True)


gonzaloa_test = pd.read_csv(
    "https://github.com/AnasElbattra/Hugging-face/raw/main/FAKE-REAL/DATA/GonzaloA_fake_news/test%20Data.csv"
)  # 16240 row
gonzaloa_test = gonzaloa_test[["text", "label"]]
# gonzaloa_test['label'].replace({0: 'fake', 1: 'real'}, inplace=True)


# pushpdeepfake_news_combined
pushpdeepfake_train = pd.read_csv(
    "https://github.com/AnasElbattra/Hugging-face/raw/main/FAKE-REAL/DATA/pushpdeepfake_news_combined/train.csv"
)  # 34916 row
pushpdeepfake_train = pushpdeepfake_train[["text", "label"]]
# clean_2_train['label'].replace({0: 'fake', 1: 'real'}, inplace=True)


pushpdeepfake_test = pd.read_csv(
    "https://github.com/AnasElbattra/Hugging-face/raw/main/FAKE-REAL/DATA/pushpdeepfake_news_combined/test.csv"
)  # 10003 row
pushpdeepfake_test = pushpdeepfake_test[["text", "label"]]
# pushpdeepfake_test['label'].replace({0: 'fake', 1: 'real'}, inplace=True)

## Trash Data

In [ ]:
# kaggle_datafake
kaggle_train = pd.read_csv(
    "https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/FAKE-REAL/DATA/kaggle_datafake/kaggle_datafake_train.csv"
)  # 24000 row
kaggle_train = kaggle_train[["text", "label"]]

kaggle_test = pd.read_csv(
    "https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/FAKE-REAL/DATA/kaggle_datafake/kaggle_datafake_test.csv"
)  # 208003 row
kaggle_test = kaggle_test[["text", "label"]]

# news_articles
news_articles = pd.read_csv(
    "https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/FAKE-REAL/DATA/news_articles.csv"
)  # 2100 row
news_articles = news_articles[["text", "label"]]
news_articles["label"] = news_articles["label"].str.lower()

## New Data

In [ ]:
# mrm8488-fake-news
mrm8488_fake_news = pd.read_csv(
    "https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/FAKE-REAL/DATA/mrm8488-fake-news/fake_news.csv"
)  # 44919 row


# ErfanMoosaviMonzzah
Erfan_train = pd.read_csv(
    "https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/FAKE-REAL/DATA/ErfanMoosaviMonazzah/train.csv"
)  # 20011 row
Erfan_train = Erfan_train[["text", "label"]]

Erfan_test = pd.read_csv(
    "https://github.com/AnasElbattra/News-Understanding-NLP/raw/main/FAKE-REAL/DATA/ErfanMoosaviMonazzah/test.csv"
)  # 24277 row
Erfan_test = Erfan_test[["text", "label"]]

## Data Combination

In [ ]:
# combine gonzaloa_train, pushpdeepfake_train
# training_data = pd.concat([gonzaloa_train, pushpdeepfake_train], ignore_index=True)
training_data = erfant_train

dataset = Dataset.from_pandas(training_data)
dataset = dataset.train_test_split(test_size=0.2)
# dataset["train"]["label"]

# Fine Tuning


In [ ]:
def to_tokens(tokenizer, label_map):
    def apply(x):
        target_labels = [label_map[y] for y in x["label"]]
        token_res = tokenizer(
            x["text"],
            text_target=target_labels,
            return_tensors="pt",
            truncation=True,
            padding=True,
        )
        return token_res

    return apply

In [ ]:
tokenizer = tr.AutoTokenizer.from_pretrained("t5-small")

In [ ]:
label_lookup = {0: "fake", 1: "real"}
to_tokens = to_tokens(tokenizer, label_lookup)
tokenized_dataset = dataset.map(
    to_tokens, batched=True, remove_columns=["text", "label"]
)

In [ ]:
local_training_root = "./"
checkpoint_name = "test-trainer"
local_checkpoint_path = os.path.join(local_training_root, checkpoint_name)
training_args = tr.TrainingArguments(
    local_checkpoint_path,
    num_train_epochs=1,  # default number of epochs to train is 3
    per_device_train_batch_size=16,
    optim="adamw_torch",
    report_to=["tensorboard"],
)

In [ ]:
t5_model = tr.AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
metric = load_metric("accuracy")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = [
        tokenizer.decode(p, skip_special_tokens=True)
        for p in predictions.argmax(dim=-1)
    ]
    labels = [label_lookup[l] for l in labels]

    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=t5_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
# trainer.evaluate()
trainer.save_model()
trainer.save_state()
checkpoint_name = "test-trainer-deepspeed"
final_model_path = f"./llm04_fine_tuning/{checkpoint_name}"

trainer.save_model(output_dir=final_model_path)

fine_tuned_model = tr.AutoModelForSeq2SeqLM.from_pretrained(final_model_path)

# Evaluation

## Data Cleaning and sampling

In [ ]:
# Take 100 samples from each dataset
# gonzaloa_train
gonzaloa_train = gonzaloa_train.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
gonzaloa_train = gonzaloa_train.sample(frac=1, random_state=42)
gonzaloa_train.dropna(subset=["text"], inplace=True)
gonzaloa_test["label"].replace({0: "fake", 1: "real"}, inplace=True)

# gonzaloa_test
gonzaloa_test = gonzaloa_test.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
gonzaloa_test = gonzaloa_test.sample(frac=1, random_state=42)
gonzaloa_test.dropna(subset=["text"], inplace=True)
gonzaloa_test["label"].replace({0: "fake", 1: "real"}, inplace=True)

# pushpdeepfake_train
pushpdeepfake_train = pushpdeepfake_train.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
pushpdeepfake_train = pushpdeepfake_train.sample(frac=1, random_state=42)
pushpdeepfake_train.dropna(subset=["text"], inplace=True)
pushpdeepfake_train["label"].replace({0: "fake", 1: "real"}, inplace=True)

# pushpdeepfake_test
pushpdeepfake_test = pushpdeepfake_test.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
pushpdeepfake_test = pushpdeepfake_test.sample(frac=1, random_state=42)
pushpdeepfake_test.dropna(subset=["text"], inplace=True)
pushpdeepfake_test["label"].replace({0: "fake", 1: "real"}, inplace=True)

# kaggle_train
kaggle_train = kaggle_train.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
kaggle_train = kaggle_train.sample(frac=1, random_state=42)
kaggle_train.dropna(subset=["text"], inplace=True)
kaggle_train["label"].replace({0: "fake", 1: "real"}, inplace=True)

# kaggle_test
kaggle_test = kaggle_test.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
kaggle_test = kaggle_test.sample(frac=1, random_state=42)
kaggle_test.dropna(subset=["text"], inplace=True)
kaggle_test["label"].replace({0: "fake", 1: "real"}, inplace=True)

# news_articles
news_articles = news_articles.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
news_articles = news_articles.sample(frac=1, random_state=42)
news_articles.dropna(subset=["text"], inplace=True)
news_articles["label"].replace({0: "fake", 1: "real"}, inplace=True)

# mrm8488_fake_news
mrm8488_fake_news = mrm8488_fake_news.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
mrm8488_fake_news = mrm8488_fake_news.sample(frac=1, random_state=42)
mrm8488_fake_news.dropna(subset=["text"], inplace=True)
mrm8488_fake_news["label"].replace({0: "fake", 1: "real"}, inplace=True)

# Erfan_test
Erfan_test = Erfan_test.groupby("label", group_keys=False).apply(
    lambda x: x.sample(min(len(x), 50))
)
Erfan_test = Erfan_test.sample(frac=1, random_state=42)
Erfan_test.dropna(subset=["text"], inplace=True)
Erfan_test["label"].replace({0: "fake", 1: "real"}, inplace=True)

In [ ]:
def evaluate_data(df):
    # Initialize an empty list to store the predictions
    predictions = []

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        text = row["text"]
        news_report = [text]  # Create a list with the text from the DataFrame

        # Tokenize and generate predictions
        inputs = tokenizer(
            news_report, return_tensors="pt", truncation=True, padding=True
        )
        pred = fine_tuned_model.generate(
            input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"]
        )

        # Decode the predictions and store them in the predictions list
        classification = tokenizer.batch_decode(pred, skip_special_tokens=True)[0]
        predictions.append(classification)

    # Add the predictions to the DataFrame as a new column
    df["predictions"] = predictions

    classification_rep = classification_report(df["label"], df["predictions"])
    print("Classification Report:")
    print(classification_rep)

    accuracy = accuracy_score(df["label"], df["predictions"])

    return accuracy

## Evaluation on the samples

In [ ]:
gonzaloa_train_accuracy = evaluate_data(gonzaloa_train)

In [ ]:
gonzaloa_test_accuracy = evaluate_data(gonzaloa_test)

In [ ]:
pushpdeepfake_train_accuracy = evaluate_data(pushpdeepfake_train)

In [ ]:
pushpdeepfake_test_accuracy = evaluate_data(pushpdeepfake_test)

In [ ]:
kaggle_train_accuracy = evaluate_data(kaggle_train)

In [ ]:
kaggle_train_accuracy = evaluate_data(kaggle_train)

In [ ]:
news_articles_accuracy = evaluate_data(news_articles)

In [ ]:
mrm8488_fake_news_accuracy = evaluate_data(mrm8488_fake_news)

In [ ]:
erfan_test_accuracy = evaluate_data(Erfan_test)

## Bar Chart

In [ ]:
import matplotlib.pyplot as plt

# Model labels
models = [
    "GonzaloA_train",
    "GonzaloA_test",
    "pushpdeepfake_train",
    "pushpdeepfake_test",
    "kaggle_train",
    "kaggle_test",
    "news_articles",
    "mrm8488_fake_news",
    "Erfan_test",
]

# Accuracy values
accuracies = [
    gonzaloa_train_accuracy,
    gonzaloa_test_accuracy,
    pushpdeepfake_train_accuracy,
    pushpdeepfake_test_accuracy,
    kaggle_train_accuracy,
    kaggle_train_accuracy,
    news_articles_accuracy,
    mrm8488_fake_news_accuracy,
    erfan_test_accuracy,
]

# Create a color list to distinguish between 'summ' and non-summ models
colors = [
    "skyblue",
    "royalblue",
    "mediumseagreen",
    "lightcoral",
    "plum",
    "gold",
    "lightpink",
    "lightgrey",
    "lightgreen",
]

# Create the bar plot
plt.figure(figsize=(10, 6))
bars = plt.bar(models, accuracies, color=colors, edgecolor="black")

# Adding value labels on top of each bar
for bar in bars:
    yval = bar.get_height()
    plt.text(
        bar.get_x() + bar.get_width() / 2,
        yval,
        round(yval, 2),
        va="bottom",
        ha="center",
    )

plt.title("Accuracy on different datasets using fine-tuned T5-small", fontsize=16)
plt.ylabel("Accuracy", fontsize=12)
plt.xlabel("Model", fontsize=12)
plt.xticks(rotation=45, fontsize=10)

# Adding a grid
plt.grid(axis="y", linestyle="--", alpha=0.7)

plt.tight_layout()
plt.show()